[<img align="left" width="140px" src="https://storage.googleapis.com/static.drivetrainhub.com/img/dh_logo_text_217x80.png"/>](https://www.drivetrainhub.com)

<div align="right">© 2019 Drivetrain Hub LLC</div>

# Gear Tooth Contact Stress
---

**Authors**:  [Tugan Eritenel](https://github.com/tugan1) 

**Description**:  Calculation of gear tooth contact stress in spur gears.

## Table of Contents

1. [Introduction](#Introduction)
    1. [Definitions and Gear Parameters](#Definitions-and-Gear-Parameters)

# Introduction
This chapter is on contact stress of parallel axis gears.  

Notebook imports and settings

In [1]:
import ipywidgets as widgets
from IPython import display
from IPython.core.display import HTML
import pandas as pd
from pprint import pprint
import math

In [2]:
def inv(a):
    return math.tan(a) - a

def alpha_at_given_d(d,d_b):
    return math.acos(d_b/d)

def thickness_at_given_d(d,d_p,d_b,alpha,m,x):
    S_p = math.pi/2*m + 2*m*x*math.tan(alpha)
    alpha_d = alpha_at_given_d(d,d_b)
    S = d*(S_p/d_p+inv(alpha)-inv(alpha_d))
    return S

# Contact Stress
Gear tooth contact along the tooth surface can be considered as a line. The width of the contact is much shorter than the length along the line of contact.  The radius of curvatures of the contacting surfaces vary as the gears rotate.  Maximum subsurface shear stress is just below the point of contact.  The mechanism of stress is quite complicated due to sliding and friction between the surfaces.  

<img src="./img/contact.png" alt="Involute parameters" style="max-width: 650px"/>
<p style="text-align: center; font-weight: bold;">Stresses on gear tooth.</p>
<span style="font-size: 8pt; float: right">Image credit: G. M. Maitra, Handbook of Gear Design </span>

Pitting is a type of gear failure as a result of subsurface contact stress cycles.  The contact stress is denoted with the symbol $p$ referring to Hertzian pressure developed.  The maximum pressure on two cylinders in contact is given by,
$$ p_{c(max)} = \frac{4 F}{\pi B L}$$ 
where the width of the contact is
$$B = \sqrt{\frac{8 F}{\pi L} \frac{\left(1-\nu_1^2\right)/E_1 + \left(1-\nu_2^2\right)/E_2}{\left(1/D_1\right) + \left(1/D_2\right)}}$$

When the elastic modulus of the two cylinders are the same $E_1 = E_2$, and poisson ratio are $\nu_1 = \nu_2 = 0.3$, the contact pressure for gears becomes,
$$ p_p = \sqrt{0.35\frac{F_t E}{b d_{p,1}} \frac{i+1}{i} \frac{1}{\cos^2\alpha \tan\alpha_w}} $$

A few parameters are factored so that
$$ p_p = y_m y_p \sqrt{\frac{F_t}{b_1 d_{p,1}} \frac{i+1}{i}}$$
where $y_m = \sqrt{0.35 E}$ becomes a material factor and $y_p = \sqrt{\frac{1}{\cos^2\alpha \tan\alpha_w}} $ becomes a tooth shape factor.  For a first design iteration, assuming no profile shift, and $\alpha = 20^{\circ}$, $y_p = 1.76$, and $E=206,000$ MPa, so $y_m = 269$.

Maximum allowable surface fatigue stress $p_{sc}$ needs to be determined from the gear material.  For example, induction hardened 40 Cr 4 allowable surface fatigue stress is 1620 MPa.  Generally a factor of safety of 1.5 is applied, giving the contact pressure limit as $p_{cp}< \frac{p_{sc}}{1.5}$.

Contact pressure is written in terms of applied torque as,
$$ p_{cp} = y_m y_p\sqrt{ \frac{2000 T_1}{\left(b/d_{p,1}\right) d_{p,1}^3} \frac{i+1}{i}}$$

Minimum module to satisfy the contact pressure safety is found using
$$ m_n < \frac{1}{z_1} \sqrt[3]{ \frac{2000 T_1 y_m^2 y_p^2}{\left(b/d_{p,1}\right) p_{cp}^2} \frac{i+1}{i}}$$

## Example
A gear pair has 45 teeth on the pinion and 137 teeth on the wheel.  Torque on the pinion is 600 Nm.  Minimum module so that the pinion is safe for contact fatigue is required.

For the first iteration of design $y_p = 1.76$ and $y_m=269$.  

The ratio of facewidth to pitch diameter is taken as $\frac{b_1}{d_{p,1}} = 0.5$ to provide a starting point for the design. The formula below is used for finding the minimum module. Substituting the given values:

In [21]:
T_1 = 600
z_1 = 45
z_2 = 137
i=z_2/z_1
y_m = 269
y_p = 1.76
facewidth_to_pitch = 0.5
p_cp = 1620/1.5


m = 1/z_1*math.pow(2000*T_1 * (y_m**2) * (y_p**2)/((facewidth_to_pitch)*p_cp**2)*(i+1)/i ,1/3)
display.display(HTML('<div class="alert alert-block alert-info"> Minimum module for bending stress, $m$ = ' + str(round(m,2)) + '</div>'))

m_nearest = math.ceil(m*2)/2
display.display(HTML('<div class="alert alert-block alert-info"> Selected module, $m_{n}$ = ' + str(round(m_nearest,2)) + '</div>'))


The contact pressure is calculated with the selected module

In [22]:
m = m_nearest
d_p1= m * z_1
F_t = 2000*T_1/d_p1
p_cp = y_m *y_p * math.sqrt( 2000*T_1 / (facewidth_to_pitch * d_p1**3)*(i+1)/i )


In [23]:
display.display(HTML('<div class="alert alert-block alert-info"> Contact pressure on pinion, $p_{cp}$ = ' + str(round(p_cp,2)) + ' MPa </div>'))
